<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2022/ex05notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2022 ex05notebookB

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2022

In [ ]:
# 必要なパッケージのインポート
import numpy as np
import pandas as pd

----
## Python + NumPy で主成分分析をやってみよう



自分で主成分分析を実行する際には，notebookA でも使っているような統計分析ツールを使うこともできますが，線形代数の授業で学んだことを活かすために，Python + NumPy で計算するコードを書いてみましょう．
以下は，そのための練習です．

---
### 固有値・固有ベクトルの計算と対称行列の対角化

notebookA で説明しているように，主成分分析のためには，与えられたデータの分散共分散行列を計算し，その固有値と固有ベクトルを求めることになります．まずは，行列の固有値・固有ベクトルを求める手順を考えます．

分散共分散行列は **対称行列** ですので，固有値はすべて実数，固有ベクトルの要素もすべて実数となります．対称行列は，固有値・固有ベクトルを使って以下のように **対角化** できるのでした．

---
［実対称行列の対角化］

$D\times D$ 行列 $A$ が対称行列である，つまり，$A^{\top} = A$ が成り立つとする．このとき，その固有値 $\lambda_{1}, \lambda_{2}, \ldots, \lambda_{D}$ はすべて実数である．
また，$\lambda_{d}$ に対応する単位固有ベクトル（大きさ $1$ の固有ベクトル）を $\mathbf{u}_{d}$ と表す（$d=1,2,\ldots, D$）とき，これらは互いに直交するように，つまり，

$$
\mathbf{u}_{i}\cdot\mathbf{u}_{j} = \left\{
    \begin{aligned}
    1 \quad & (i = j)\\
    0 \quad & (i \ne j)
    \end{aligned}
\right. \qquad (D1)
$$

となるようにとることができる（$i, j = 1, 2, \ldots, D$）．

このとき，固有値固有ベクトルの定義から $A\mathbf{u}_{d} = \lambda_{d}\mathbf{u}_{d}$ なので，

$$
A \left(\mathbf{u}_{1}\ \mathbf{u}_{2}\ \cdots \mathbf{u}_{D} \right) = \left(\lambda_{1}\mathbf{u}_{1}\ \lambda_{2}\mathbf{u}_{2}\ \cdots \lambda_{D}\mathbf{u}_{D} \right) = \left(\mathbf{u}_{1}\ \mathbf{u}_{2}\ \cdots \mathbf{u}_{D} \right)
\begin{pmatrix}
\lambda_{1} & & &\\
 & \lambda_{2} & &  \\
& &\ddots & \\
& & & \lambda_{D} \\
\end{pmatrix}
\qquad (D2)
$$

となる．$\mathbf{u}_{d}$ を列ベクトル（$D\times 1$行列）として，$D\times D$ 行列 $U$ を $U = \left(\mathbf{u}_{1}\ \mathbf{u}_{2}\ \cdots \mathbf{u}_{D} \right)$ とおくと，式($D1$)より，

$$
U^{\top}U = 
\begin{pmatrix}
\mathbf{u}_{1}^{\top} \\ \mathbf{u}_{2}^{\top} \\ \vdots \\ \mathbf{u}_{D}^{\top}
\end{pmatrix}
\left(\mathbf{u}_{1}\ \mathbf{u}_{2}\ \cdots \mathbf{u}_{D} \right)
=
\begin{pmatrix}
\mathbf{u}_{1}^{\top}\mathbf{u}_{1} & \mathbf{u}_{1}^{\top}\mathbf{u}_{2} & \cdots & \mathbf{u}_{1}^{\top}\mathbf{u}_{D}\\
\mathbf{u}_{2}^{\top}\mathbf{u}_{1} & \mathbf{u}_{2}^{\top}\mathbf{u}_{2} & \cdots & \mathbf{u}_{2}^{\top}\mathbf{u}_{D} \\
\vdots  & \vdots & \vdots & \vdots \\
\mathbf{u}_{D}^{\top}\mathbf{u}_{1} & \mathbf{u}_{D}^{\top}\mathbf{u}_{2} & \cdots & \mathbf{u}_{D}^{\top}\mathbf{u}_{D} \\
\end{pmatrix}
=
\begin{pmatrix}
1 & & &\\
 & 1 & &\\
& & \ddots &\\
 & & & 1
\end{pmatrix}
= I
$$

となるので，式(D2)に左から $U^{\top}$ をかけて，

$$
U^{\top}AU = U^{\top}U 
\begin{pmatrix}
\lambda_{1} & & &\\
 & \lambda_{2} & &  \\
& &\ddots & \\
& & & \lambda_{D} \\
\end{pmatrix}
= 
\begin{pmatrix}
\lambda_{1} & & &\\
 & \lambda_{2} & &  \\
& &\ddots & \\
& & & \lambda_{D} \\
\end{pmatrix}
$$

が得られる．つまり，実対称行列 $A$ の固有ベクトルをならべた行列を $U$ とするとき，$A$ の左から $U^{\top}$，右から $U$ を掛けると，対角要素に固有値が並んだ対角行列となる．

---

これをふまえて，適当に作った対称行列の固有値・固有ベクトルを求めて，本当に対角化できることを計算で確かめてみましょう．

In [ ]:
# 3x3 対称行列
A = np.array([[5, 3, -1], [3, 4, 1], [-1, 1, 2]])
print('A = ')
print(A, A.shape)

次のセルを実行すると，`A` の固有値・固有ベクトルを求めて表示します．
ここでは，NumPy で「特異値分解」を行う `np.linalg.svd` という関数を使っています（注）．


<span style="font-size: 75%">
※注: <a href="https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html">np.linalg.svd</a>．<a href="https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html">np.linalg.eig</a> という関数もありますが，そちらは固有値が順番に並ばないのでここでは使っていません．
ちなみに，固有値・固有ベクトルを使った対角化（固有値分解）は正方行列にしか適用できませんが，「特異値分解」は行数と列数が等しくない行列でも対角化もどきの分解ができます．
</span>


In [ ]:
## 特異値分解経由で A の固有値・固有ベクトルを求める
#
# np.linalg.svd は「特異値分解」を行う関数．
#    引数にわたす行列 A が実対称行列の場合，固有値分解と等価な結果が得られる
#    固有値（特異値）は降順に並ぶ
U, eval, Vt = np.linalg.svd(A)
print('U = ')
print(U, U.shape)
print('eval = ')
print(eval, eval.shape)
print()
for d in range(len(eval)): # 固有ベクトルは列ベクトルとして U に格納されてる
    print(f'固有値 {eval[d]:.2f} に対応する固有ベクトルは {U[:, d]}')
print()
print('U.T @ U = ')
print(U.T @ U)

`eval` の中に3つの固有値が降順に格納されています．`U` の方には，固有ベクトルが「列ベクトルとして」格納されています．つまり，1列が一つの固有ベクトルです（「1行」ではないことに注意）．したがって，`eval[d]` が `A` の（0から数えて）`d`番目に大きな固有値，`U[:, d]` がそれに対応する固有ベクトルです．

`U.T @ U` のところは，$U^{\top}U = I$ となっていることを確認しています．

#### 問題1

1. 上記を読んで理解しましょう．
1. `U.T @ U` の値が本当に↑の通りになっているか自分の目で確かめましょう．

---
### 行列・ベクトルの扱いに関する注意



notebookA の説明では，$D$ 次元のデータを $H$ 次元にする線形変換を説明する際に，次の式を考えていました（ここでは，簡単のため，$D = 3, H = 2$ で書いてます）．

$$
\begin{pmatrix}
    y_1\\y_2
\end{pmatrix}
= 
\begin{pmatrix}
w_{1,1} & w_{1,2} & w_{1,3}\\
w_{2,1} & w_{2,2} & w_{2,3}
\end{pmatrix}
\begin{pmatrix}
    x_1\\x_2\\x_3
\end{pmatrix}
$$

$y_1, y_2$ を並べた $2\times 1$ 行列を $\mathbf{y}$，$w_{1,1}, \ldots, w_{2,3}$ を並べた $2\times 3$ 行列を $W$，$x_{1}, x_{2}, x_{3}$ を並べた $3\times 1$ 行列を $\mathbf{x}$ をおくと， この式は次のように書けます．

$$
\mathbf{y} = W\mathbf{x} \qquad (M1)
$$

線形代数の教科書等では，このように，$D$ 次元のベクトルを「列ベクトル」（$D\times 1$行列）として扱うのが標準的です．そのため，notebookA の説明もそちらで書きました．



一方，コンピュータでデータを扱う際は，「行ベクトル」で考えることがよくあります（
理由は，文字の入出力の際に，1行目を左から右へ向かって書く → 2行目で同様 → ...と進むので，1列ではなく1行がひとかたまりの方が便利ってところでしょうか）．
重回帰分析の項では実際にそのように数値を並べた配列を扱ってきました．行列として書くと次のようなものです（$D=3$の例を書いています）．

$$
X =
\begin{pmatrix}
x_{1, 1} & x_{1, 2} & x_{1, 3}\\
x_{2, 1} & x_{2, 2} & x_{2, 3}\\
\vdots & \vdots & \vdots\\
x_{N, 1} & x_{N, 2} & x_{N, 3}\\
\end{pmatrix}
$$


この場合，一つのデータ $\mathbf{x}_{n} = (x_{n, 1}, x_{n, 2}, x_{n, 3})$ は「行ベクトル」（$1\times D$行列）として `X` に格納されています．



このとき，$\mathbf{x}_{n}$ に行列 $W$ をかけて $H\times 2$ 行列 $\mathbf{y}_{n} = (y_{n,1}, y_{n,2})$ を求める計算は，次式のようになります（ちょうど式(1)の両辺を転置したものになってますね）．

$$
\mathbf{y}_{n} = \mathbf{x}_{n}W^{\top}
$$

したがって，上記の並び順で数値が格納されている $N\times D$ 行列 $X$ に $H\times D$ 行列 $W$ をかけて $N$ 個の $H$ 次元ベクトルを求める計算は次式のようになります．

$$
Y = XW^{\top} \qquad (M2)
$$

#### 問題2

問題1の方で求めた固有ベクトルを使って，式(M2)の計算を実際にやってみましょう．

In [ ]:
X = np.array([[1, 1, 1], [1, 0, -1], [0, 2, 2], [-2, -2, -2]])
print('X = ')
print(X, X.shape)

(1) `X` が上の説明の行列 $X$ を表していると考えると，$N, D$ はそれぞれいくつか答えなさい．

(2) 次のセルの1行目を修正して，`W` の1**行**目が `A` の最大固有値に対応した固有ベクトルに，2**行**目が `A` の2番目に大きな固有値に対応した固有ベクトルになるようにしなさい．

ヒント: 配列のスライスを使いましょう．`X[:2, :]` とか `X[:, :2]` とかの値を出力させてみたらヒントになるでしょう．

In [ ]:
W = U
print('W = ')
print(W, W.shape)

(3) 次のセルの1行目を修正して，`Y` が式(M2)で表されるものになるようにしなさい．
次のようになるはずです．

```
Y = 
[[-1.38606979 -0.87366507]
 [-0.7885154   1.16663307]
 [-1.23980944 -2.53977196]
 [ 2.77213957  1.74733013]] (4, 2)
 ```

In [ ]:
Y = X
print('Y = ')
print(Y, Y.shape)

### データの平均を0にする，分散共分散行列を求める

データから分散共分散行列を求める過程を考えましょう．

上で説明してきたデータに対して，分散共分散行列 $V$は，その $(i,j)$ 要素 $V_{i,j}$ が

$$
V_{i,j} = \frac{1}{N}\sum_{n=1}^{N}(x_{n,i}-\bar{x}_{n,i})(x_{n,j}-\bar{x}_{n,j})  \qquad (i, j = 1, 2, \ldots, D)
$$

となるような $D\times D$ 行列ですが，$\frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_{n} = \mathbf{0}$ である，つまりデータの平均が $\mathbf{0}$ である場合には，

$$
V_{i,j} = \frac{1}{N}\sum_{n=1}^{N}x_{n,i}x_{n,j}  \qquad (i, j = 1, 2, \ldots, D)
$$

で計算することができます．


上で説明してきたようにデータが $N \times D$ 行列 $X$ に格納されている場合，$1\times D$ 行列 $\mathbf{x}_n = \begin{pmatrix}
x_{n,1} & x_{n,2} & \cdots & x_{n,D}
\end{pmatrix}
$
として，次式のように表せます．

$$
X =
\begin{pmatrix}
x_{1, 1} & x_{1, 2} & x_{1, 3}\\
x_{2, 1} & x_{2, 2} & x_{2, 3}\\
\vdots & \vdots & \vdots\\
x_{N, 1} & x_{N, 2} & x_{N, 3}\\
\end{pmatrix}
=
\begin{pmatrix}
\mathbf{x}_{1}\\
\mathbf{x}_{2}\\
\vdots\\
\mathbf{x}_{N}
\end{pmatrix}
$$

このとき，

$$
X^{\top}X =
\begin{pmatrix}
\mathbf{x}_{1}^{\top} &
\mathbf{x}_{2}^{\top} &
\cdots &
\mathbf{x}_{N}^{\top}
\end{pmatrix}
\begin{pmatrix}
\mathbf{x}_{1}\\
\mathbf{x}_{2}\\
\vdots\\
\mathbf{x}_{N}
\end{pmatrix}
=
\sum_{n=1}^{N}\mathbf{x}_{n}^{\top}\mathbf{x}_{n}
$$

の $(i,j)$ 要素は

$$
\sum_{n=1}^{N}x_{n,i}x_{n,j}  \qquad (i, j = 1, 2, \ldots, D)
$$

となっています．
したがって，$N \times D$ 行列 $X$ に格納されている，平均が $\mathbf{0}$ のデータの分散共分散行列 $V$ は，$V = \frac{1}{N}X^{\top}X$ と表せます．



#### 問題3

実際のデータから分散共分散行列を求めてみましょう．

In [ ]:
# 国数英
dfJME = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/jme.txt', delimiter=' ', header=None)
dfJME.rename(columns={0:'国語', 1:'数学', 2:'英語'}, inplace=True)
dfJME.head(5)

In [ ]:
Xorg = dfJME.to_numpy()
print(Xorg)
N, D = Xorg.shape  # Xorg の行数が N に，列数が D に代入される
print(f'N = {N}, D = {D}')

(1) 次のセルに，`np.mean` を使って `Xorg` の列ごとの平均を求めてそれを変数 `Xm` に代入し，その値を表示するコードを書きなさい（ヒント: `axis`オプションを使う）．
こうなるはず．
```
[58.13  61.8  67.3]
```

(2) 次のセルに，`Xorg` から `Xm` を引いたものを `X` に代入し，その値を表示するコードを書きなさい．

(3) 次のセルに，`Xorg` に格納されていたデータの分散共分散行列を求めて `V` に代入するコードを書きなさい．`X` と `N` を使うこと．

こうなるはず：
- 国語，数学，英語の分散はそれぞれ 60.45, 68.5, 53.6
- 数学と英語の共分散は 50.9，国語と英語の共分散は 46.3，国語と数学の共分散は 33.6

### 固有値・固有ベクトルを求める，次元削減する



#### 問題4

上のデータを主成分分析によって2次元にする次元削減をやってみましょう．


(1) 次のセルに，問題3で求めた `V` を使ってその固有値・固有ベクトルを求め，それらの値を表示するコードを書きなさい．

(2) 次のセルに，`X` に格納されたデータを2次元に次元削減する（第1，第2主成分スコアを求める）コードを書きなさい．
ただし，次の指示にしたがうこと
- `V` の固有ベクトルのうち大きい方から2つを0行目と1行目に並べたものを `W` としなさい
- `W` を使って `X` を変換したものを `Y` としなさい

`Y` の最初の行と最後の行はこんなんなるでしょう．
```
[ 16.31671919   0.44369665]
[ -7.49938384  10.01579253]
```

### 主成分分析の結果を解釈する

主成分分析の結果をどのように分析するか，については次回説明しますが，ここでその初歩を少しだけ．

#### 問題5

次のようなデータをでっち上げて考えてみましょう．

In [ ]:
X2 = np.array([[  0,   0,   0], # 3科目とも平均点のひと
               [ 10,  10,  10], # 3科目とも平均より10点上だったひと
               [ 20,  20,  20], # 3科目とも平均より20点上だったひと
               [-10, -10, -10], # 3科目とも平均より10点下だったひと
               [ 10,  10,   0], # 国語数学が等しく平均より上だったひと
               [ 10, -10,   0], # 国語が平均より上で数学が平均より下
               [-10,  10,   0], # 国語が平均より下で数学が平均より上
               [-20,   0,   0], # 国語が平均より下で苦手で数学，英語は平均
])

(1) 次のセルに，問題3の `W` を使って `X2` を 2次元に変換したものを求め，その値を表示するコードを書きなさい．変換したものは `Y2` という変数に代入することにしよう．

`X2` と `Y2` を見比べると，次のようなことが分かります．
- 3科目とも平均点のひとの主成分スコアは $(\mbox{第1主成分スコア}, \mbox{第2主成分スコア} ) = (0, 0)$ となっている
- 3科目どれも平均より上のひとの第1主成分スコアは負，3科目とも平均より下のひとは正
- 国語と数学で国語の方がよいひとは第2主成分スコアが正，逆のひとは第2主成分スコアが負

このような結果が得られることは，`W` の要素からも推測できます．

In [ ]:
print(W)

`W`の 0 行目，すなわちデータの分散共分散行列の最大固有値に対応する固有ベクトルは3つの要素がすべて負で値の大きさがほぼ同じで，おおざっぱに見ると

$$
(\mbox{第1主成分}) = -0.5(\mbox{国語}) -0.5(\mbox{数学}) -0.5(\mbox{英語}) = -0.5((\mbox{国語})+(\mbox{数学})+(\mbox{英語}))
$$

となっています．このことから，第1主成分は「3科目の総合力」（ただし値が小さい方が高い）を表すと解釈できます．

一方，第2主成分については，(英語)の係数が小さく，おおざっぱには

$$
(\mbox{第1主成分}) = 0.7(\mbox{国語}) -0.7(\mbox{数学}) = 0.7((\mbox{国語})-(\mbox{数学}))
$$

と見ることができます．したがって，第2主成分は「国語の得意度合い」（大きいほうが数学に比べて国語が得意）とでもいえそうです．最後の二人は国数の点数は異なるけれど，差は同じなので，第2主成分スコアの値はほとんど同じになっています．

これらのことから，ここで行った主成分分析による次元削減では，国語，数学，英語3科目の点数のデータを2つの変数で説明しようとしたら，「3科目の総合力」と「国語力」という2つが得られた，この二つがこのデータの主な成分である，という解釈をすることができます．